# CSV And Excel files- Structured Data

In [1]:
import pandas as pd
import os

## Create Sample Data

In [2]:
os.makedirs("data/structured_files", exist_ok=True)

In [3]:
# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

# Save as CSV
df = pd.DataFrame(data)
df.to_csv('data/structured_files/products.csv', index=False)

In [4]:
# Save as Excel with multiple sheets
with pd.ExcelWriter('data/structured_files/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)
    
    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

## CSV Processing

In [5]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

### METHOD 1: CSVLoader - Each row becomes a document

In [6]:

# -------------------------------------------------
# This method uses LangChain’s `CSVLoader` to load structured CSV data.
# Each row in the CSV file becomes a separate `Document` object.

print("1️⃣ CSVLoader - Row-based Documents")

# Create an instance of CSVLoader
csv_loader = CSVLoader(
    # Path to your CSV file
    file_path='data/structured_files/products.csv',  
    # Character encoding for reading text
    encoding='utf-8',     
    # Optional arguments passed to Python's CSV reader                          
    csv_args={        
        # Field separator (e.g., ',' or ';')                              
        'delimiter': ',',               
        # Character used for quoting text fields            
        'quotechar': '"',                           
    }
)

# Load the data from the CSV file.
# Each row is converted into a LangChain `Document` object.
# - page_content → combined text representation of that row
# - metadata → column names and row index (or file info)
csv_docs = csv_loader.load()

# Print all loaded documents (for inspection)
print(csv_docs)

# Show how many rows were loaded
print(f"Loaded {len(csv_docs)} documents (one per row)")

# Display the content of the first document for a quick preview
print("\nFirst document:")
# Shows the text content from the first row
print(f"Content: {csv_docs[0].page_content}") 
# Shows metadata such as column headers and file path 
print(f"Metadata: {csv_docs[0].metadata}")     


1️⃣ CSVLoader - Row-based Documents
[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 4}, page_content

### Method 2: Custom CSV Processing for Better Control

In [7]:
from typing import List
from langchain_core.documents import Document
import pandas as pd   # Required for DataFrame operations

# ------------------------------------------------------
# Method 2: Custom CSV Processing for Better Control
# ------------------------------------------------------
print("\n2️⃣ Custom CSV Processing")

def process_csv_intelligently(filepath: str) -> List[Document]:
    """
    Reads a CSV file and converts it into a list of LangChain Document objects,
    where each document corresponds to a row, but with structured and readable formatting.
    """

    # Step 1: Load the CSV file into a pandas DataFrame
    # Pandas provides rich data handling capabilities, allowing column-level access and transformations.
    df = pd.read_csv(filepath)

    # Step 2: Initialize an empty list to hold Document objects
    documents = []
    
    # Step 3: Iterate over each row in the DataFrame using iterrows()
    # This allows access to both the index (row number) and data (as a Series).
    for idx, row in df.iterrows():

        # Step 4: Construct a readable, structured text block for the LLM.
        # This ensures the model receives clear context and semantically separated attributes.
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""

        # Step 5: Create a LangChain Document object
        # The `page_content` holds the readable text,
        # while the `metadata` holds structured data for search, filtering, or indexing.
        doc = Document(
            page_content=content,
            metadata={
                # The file path for traceability
                'source': filepath,     
                # The row index for reference     
                'row_index': idx,            
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                # Custom tag to identify this document type
                'data_type': 'product_info'  
            }
        )

        # Step 6: Append the new Document to the list
        documents.append(doc)

    # Step 7: Return the list of processed Document objects
    return documents



2️⃣ Custom CSV Processing


In [8]:
process_csv_intelligently('data/structured_files/products.csv')

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_type': 'product_info'}, page_content='Product Informa

### Comparison

| Feature / Criteria                   | **Method 1 — CSVLoader**                      | **Method 2 — Custom CSV Processing**                  |
| ------------------------------------ | ------------------------------------------------ | -------------------------------------------------------- |
| **Library Used**                     | `langchain_community.document_loaders.CSVLoader` | `pandas` + manual `Document` creation                    |
| **Setup Complexity**                 | Very simple (few lines)                        | Moderate (custom function)                            |
| **Dependencies**                     | Minimal (no pandas needed)                       | Requires `pandas`                                        |
| **Performance (Speed)**              | Fast (streamed parsing)                        | Slightly slower (DataFrame overhead)                  |
| **Structure Handling**               | Flat text (all columns concatenated)           | Structured, formatted, and readable                    |
| **Metadata Richness**                | Basic (source file, row index)                | Rich (custom metadata: product, category, etc.)       |
| **Custom Formatting**                | Not supported                                  | Full control (labels, multiline text, etc.)            |
| **Readability of Output**            | Raw and compressed                             | Human-friendly and LLM-optimized                       |
| **Scalability (Large Files)**        | Efficient for big CSVs                         | Uses more memory (due to pandas)                      |
| **Error Handling**                   | Basic                                            | Can add validation & preprocessing                       |
| **Use in RAG Pipelines**             | Works but limited context                     | Ideal — clear semantic structure                       |
| **Filtering / Querying by Metadata** | Minimal                                        | Easy — rich metadata fields                            |
| **Ideal For**                        | Quick ingestion of simple CSVs                   | Intelligent, structured data loading for retrieval tasks |


In [9]:
# 📊 CSV Processing Strategies
print("\n📊 CSV Processing Strategies:")
print("\n1. Row-based (CSVLoader):")
print("  ✅ Simple one-row-one-document")
print("  ✅ Good for record lookups")
print("  ❌ Loses table context")

print("\n2. Intelligent Processing:")
print("  ✅ Preserves relationships")
print("  ✅ Creates summaries")
print("  ✅ Rich metadata")
print("  ✅ Better for Q&A")


📊 CSV Processing Strategies:

1. Row-based (CSVLoader):
  ✅ Simple one-row-one-document
  ✅ Good for record lookups
  ❌ Loses table context

2. Intelligent Processing:
  ✅ Preserves relationships
  ✅ Creates summaries
  ✅ Rich metadata
  ✅ Better for Q&A


## Excel Processing

###  METHOD 1: Using pandas for full control

In [10]:
# METHOD 1: Using pandas for full control
print("Pandas-based Excel Processing")

from typing import List
from langchain_core.documents import Document
import pandas as pd

def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel file intelligently — one Document per sheet with metadata."""
    documents = []
    
    # Step 1: Load the Excel file
    # Using `pd.ExcelFile()` gives you access to all sheet names efficiently.
    excel_file = pd.ExcelFile(filepath)
    
    # Step 2: Iterate through each sheet in the Excel workbook
    for sheet_name in excel_file.sheet_names:
        # Read the current sheet into a DataFrame
        df = pd.read_excel(filepath, sheet_name=sheet_name)
        
        # Step 3: Build readable content for each sheet
        # This combines sheet name, structure info, and the data table itself.
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)  # Convert DataFrame to readable text
        
        # Step 4: Create a Document for the sheet
        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': filepath,          # File path for traceability
                'sheet_name': sheet_name,    # The Excel sheet name
                'num_rows': len(df),         # Row count
                'num_columns': len(df.columns), # Column count
                'data_type': 'excel_sheet'   # Custom metadata tag
            }
        )
        
        # Step 5: Add the Document to the list
        documents.append(doc)
    
    # Step 6: Return all documents (one per sheet)
    return documents


Pandas-based Excel Processing


In [11]:
excel_docs = process_excel_with_pandas('data/structured_files/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")

Processed 2 sheets


In [12]:
excel_docs

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5, 'data_type': 'excel_sheet'}, page_content='Sheet: Products\nColumns: Product, Category, Price, Stock, Description\nRows: 5\n\n Product    Category  Price  Stock                                         Description\n  Laptop Electronics 999.99     50 High-performance laptop with 16GB RAM and 512GB SSD\n   Mouse Accessories  29.99    200        Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150           Mechanical keyboard with RGB backlighting\n Monitor Electronics 299.99     75                 27-inch 4K monitor with HDR support\n  Webcam Electronics  89.99    100                1080p webcam with noise cancellation'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3, 'data_type': 'excel_sheet'}, page_content='Sheet: Summary\nColumns: Category, Total_Ite

### Method 2 : UnstructuredExcelLoader

In [29]:
from langchain_community.document_loaders import UnstructuredExcelLoader

print("\nUnstructuredExcelLoader")

try:
    excel_loader = UnstructuredExcelLoader(
        'data/structured_files/inventory.xlsx',
        mode="elements"  # "elements" gives fine-grained structure; "single" merges into one doc
    )
    unstructured_docs = excel_loader.load()
    print("  Handles complex Excel features")
    print("  Preserves formatting info")
    print("  Requires unstructured library")
except Exception as e:
    print("  Requires unstructured library with Excel support")



UnstructuredExcelLoader
  Handles complex Excel features
  Preserves formatting info
  Requires unstructured library


In [30]:
unstructured_docs

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'file_directory': 'data/structured_files', 'filename': 'inventory.xlsx', 'last_modified': '2025-10-19T11:27:55', 'page_name': 'Products', 'page_number': 1, 'text_as_html': '<table><tr><td>Product</td><td>Category</td><td>Price</td><td>Stock</td><td>Description</td></tr><tr><td>Laptop</td><td>Electronics</td><td>999.99</td><td>50</td><td>High-performance laptop with 16GB RAM and 512GB SSD</td></tr><tr><td>Mouse</td><td>Accessories</td><td>29.99</td><td>200</td><td>Wireless optical mouse with ergonomic design</td></tr><tr><td>Keyboard</td><td>Accessories</td><td>79.99</td><td>150</td><td>Mechanical keyboard with RGB backlighting</td></tr><tr><td>Monitor</td><td>Electronics</td><td>299.99</td><td>75</td><td>27-inch 4K monitor with HDR support</td></tr><tr><td>Webcam</td><td>Electronics</td><td>89.99</td><td>100</td><td>1080p webcam with noise cancellation</td></tr></table>', 'languages': ['eng'], 'filetype': 'applicatio